In [41]:
import glob
import matplotlib.pyplot as plt
import os
import re
import imageio

In [42]:
# Inputs
transaction_ID = '200008'


In [43]:
# Function to extract the numeric value from filenames
def extract_row_col(filepath):
    match = re.findall(r'(\d+)', os.path.basename(filepath))  # Find all numbers in filename
    if len(match) >= 3:
        return (int(match[1]), int(match[2]))  # Use second and third numbers (row, col)
    return (9999, 9999)  # Default large number to push problematic files to the end

# Get all the input images and sort them numerically
folder_data_input = sorted(
    glob.glob(os.path.join(os.path.expanduser("~"), "satellite-ml-solarp-detection","acquisition", transaction_ID, "*tiff")),
    key= extract_row_col
)

folder_data_preds = sorted(
    glob.glob(os.path.join(os.path.expanduser("~"), "satellite-ml-solarp-detection","prediction", transaction_ID, "*tiff")),
    key= extract_row_col
)


In [44]:
input_image_paths = folder_data_input[:]
preds_image_paths = folder_data_preds[:]

print(input_image_paths)
print(120*"-")
print(f'Number of files in the acquisition ID "{transaction_ID}": {len(input_image_paths)}')


['/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_000_000.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_000_001.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_000_002.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_000_003.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_001_000.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_001_001.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_001_002.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_001_003.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_002_000.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/200008/200008_002_001.tiff', '/home/sagemaker-user/satellite-ml-solarp-detection/acquisition/20000

In [50]:
# Determine the the grid size
nb_of_rows = int(input_image_paths[-1][-12:-9]) + 1
nb_of_cols = int(input_image_paths[-1][-8:-5]) + 1

print(f'Grid shape: {nb_of_rows} rows by {nb_of_cols} cols.')



Grid shape: 4 rows by 4 cols.


In [51]:
# Generate Report
fig, axes = plt.subplots(2, nb_of_rows, nb_of_cols, figsize=(20, 10), squeeze=False)

for idx, (input_path, pred_path) in enumerate(zip(input_image_paths, preds_image_paths)):

    row = idx // nb_of_rows
    col = idx % nb_of_cols
    
    input_img = imageio.v2.imread(input_path)
    pred_img = imageio.v2.imread(pred_path)

    # Normalize input images for display
    input_img = (input_img - input_img.min()) / (input_img.max() - input_img.min()) if input_img.max() > 1 else input_img
    
    axes[0, row, col].imshow(input_img)
    axes[0, row, col].set_title(f"Input {idx+1}")
    axes[0, row, col].axis("off")
    
    axes[1, row, col].imshow(pred_img, cmap="gray")
    axes[1, row, col].set_title(f"Prediction {idx+1}")
    axes[1, row, col].axis("off")

plt.tight_layout()
plt.show()




TypeError: subplots() takes from 0 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [38]:

# # Compute Statistics
# def compute_statistics(predicted_paths):
#     stats = []
#     for path in predicted_paths:
#         pred_img = imageio.imread(path)
#         total_pixels = pred_img.size
#         positive_pixels = np.sum(pred_img > 0)
#         coverage_percentage = (positive_pixels / total_pixels) * 100
#         stats.append({
#             "File": os.path.basename(path),
#             "Total Pixels": total_pixels,
#             "Positive Pixels": positive_pixels,
#             "Coverage %": coverage_percentage
#         })
#     return pd.DataFrame(stats)

# stats_df = compute_statistics(predicted_image_paths)
# print(stats_df)

# # Save the report as a CSV
# report_path = os.path.join(os.path.expanduser("~"), "satellite-ml-solarp-detection", "reports", f"report_{transaction_ID}.csv")
# os.makedirs(os.path.dirname(report_path), exist_ok=True)
# stats_df.to_csv(report_path, index=False)
# print(f"Report saved: {report_path}")


NameError: name 'predicted_image_paths' is not defined